In [26]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import ResNet18_Weights, resnet18

In [14]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
train_dataset = datasets.CIFAR10(
    root='./data', 
    train=True, 
    download=True, 
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

val_dataset = datasets.CIFAR10(
    root='./data', 
    train=False, 
    download=True, 
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.backbone = nn.Sequential(*list(self.model.children())[:-1])
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(512, 256)
        self.relu = nn.LeakyReLU()
        self.norm = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.2)
        self.l2 = nn.Linear(256, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.flatten(x)
        x = self.l1(x)
        x = self.relu(x)
        x = self.norm(x)
        x = self.dropout(x)
        x = self.l2(x)
        x = self.softmax(x)
        return x

In [17]:
from torch.optim import Adam
import time

model = Net()
model = model.to(device)

cost_func = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

def train(train_loader, model, optimizer, cost_function):
    true_predicts, total_loss = 0, 0
    
    model.train()
    for i, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        pred = model(data)
        preds_label = torch.argmax(pred, dim=1)
        loss = cost_function(pred, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        true_predicts += torch.sum(preds_label == target).item()
        total_loss += loss.item()
    
    accuracy = true_predicts / len(train_loader.dataset)
    loss = total_loss / len(train_loader)
    
    return accuracy, loss


def validate(test_loader, model, cost_function):
    true_predicts, total_loss = 0, 0
    
    model.eval()
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        
        with torch.no_grad():
            pred = model(data)
            loss = cost_function(pred, target)
        
        preds_label = torch.argmax(pred, dim=1)        
        true_predicts += torch.sum(preds_label == target).item()
        total_loss += loss.item()
    
    accuracy = true_predicts / len(test_loader.dataset)
    loss = total_loss / len(test_loader)
    
    return accuracy, loss

EPOCHS = 10
min_loss = float('inf')

for epoch in range(EPOCHS):
    start = time.time()
    
    train_acc, train_loss = train(train_loader, model, optimizer, cost_func)
    val_acc, val_loss = validate(val_loader, model, cost_func)
    
    end = time.time()
    time_taken = end - start
    
    print(f'Epoch: {epoch+1}/{EPOCHS} --> Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f} | Time taken: {time_taken:.4f}')
    
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), 'best_model_weights.pt')

Epoch: 1/10 --> Train Loss: 1.7690, Train Accuracy: 0.6967 | Val Loss: 1.7238, Val Accuracy: 0.7390 | Time taken: 190.7367
Epoch: 2/10 --> Train Loss: 1.6801, Train Accuracy: 0.7825 | Val Loss: 1.6675, Val Accuracy: 0.7937 | Time taken: 234.3184
Epoch: 3/10 --> Train Loss: 1.6500, Train Accuracy: 0.8119 | Val Loss: 1.6854, Val Accuracy: 0.7752 | Time taken: 211.4378
Epoch: 4/10 --> Train Loss: 1.6278, Train Accuracy: 0.8336 | Val Loss: 1.6304, Val Accuracy: 0.8304 | Time taken: 227.2771
Epoch: 5/10 --> Train Loss: 1.6127, Train Accuracy: 0.8485 | Val Loss: 1.6422, Val Accuracy: 0.8199 | Time taken: 243.7405
Epoch: 6/10 --> Train Loss: 1.6025, Train Accuracy: 0.8585 | Val Loss: 1.6232, Val Accuracy: 0.8384 | Time taken: 235.5453
Epoch: 7/10 --> Train Loss: 1.5880, Train Accuracy: 0.8734 | Val Loss: 1.6080, Val Accuracy: 0.8532 | Time taken: 251.4652
Epoch: 8/10 --> Train Loss: 1.5830, Train Accuracy: 0.8778 | Val Loss: 1.6096, Val Accuracy: 0.8502 | Time taken: 171.6313
Epoch: 9/10 --> 

In [25]:
def evaluate(val_loader, model, cost_function):
    true_predicts, total_loss = 0, 0
    
    model.eval()
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        
        with torch.no_grad():
            pred = model(data)
            loss = cost_function(pred, target)
        
        preds_label = torch.argmax(pred, dim=1)        
        true_predicts += torch.sum(preds_label == target).item()
        total_loss += loss.item()
    
    accuracy = true_predicts / len(val_loader.dataset)
    loss = total_loss / len(val_loader)
    
    return accuracy, loss


# best_model = Net()
best_model = Net()
best_model.load_state_dict(torch.load('best_model_weights.pt', weights_only=True))
best_model.to(device)

accuracy, loss = evaluate(val_loader, best_model, cost_func)
print(f'Accuracy: {accuracy:.4f} | Loss: {loss:.4f}')

Accuracy: 0.8630 | Loss: 1.5983
